# Import packages

In [15]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression
from neural_data_analysis.neural_analysis_tools.align_trials import align_trial_utils
from decision_making_analysis.compare_GUAT_and_TAFT import find_GUAT_or_TAFT_trials

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_psth import core_stops_psth, get_stops_utils, psth_postprocessing, psth_stats, compare_events, dpca_utils
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_fit import stop_glm_fit, cv_stop_glm, glm_fit_utils, variance_explained
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_plotting import plot_spikes, plot_glm_fit, plot_tuning_func
from neural_data_analysis.design_kits.design_around_event import event_binning, stop_design, cluster_design, design_checks
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_hyperparams import compare_glm_configs, glm_hyperparams_class
from neural_data_analysis.topic_based_neural_analysis.ff_visibility import ff_vis_epochs, vis_design
from neural_data_analysis.neural_analysis_tools.glm_tools.glm_decoding_tools import glm_decoding_llr, glm_decoding


import sys
import math
import gc
import subprocess
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import cProfile
import pstats

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr
import statsmodels.api as sm

# Neuroscience specific imports
import neo
import rcca

# To fit gpfa
import numpy as np
from importlib import reload
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
from elephant.gpfa import GPFA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from elephant.gpfa import gpfa_core, gpfa_util

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

print("done")


%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
done
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# retrieve data

In [32]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0413"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0321"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0329"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0403"

In [33]:
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0312"

# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0316"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0327"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0328"

In [34]:
reduce_y_var_lags = False
planning_data_by_point_exists_ok = True
y_data_exists_ok = True

pn = pn_aligned_by_event.PlanningAndNeuralEventAligned(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)

if not hasattr(pn, 'spikes_df'):
    pn.retrieve_or_make_monkey_data()
    pn.spikes_df = neural_data_processing.make_spikes_df(pn.raw_data_folder_path, pn.ff_caught_T_sorted,
                                                            sampling_rate=pn.sampling_rate)

Loaded binned_spikes_df from all_monkey_data/processed_neural_data/monkey_Bruno/data_0312/binned_spikes_df_0p05.csv
Loaded planning_data_by_point from all_monkey_data/planning_and_neural/monkey_Bruno/data_0312/planning_data_by_point/opt_arc_stop_closest/test/cur_vis_0_1_window_-25cm_0cm
Loaded planning_data_by_point from all_monkey_data/planning_and_neural/monkey_Bruno/data_0312/planning_data_by_point/opt_arc_stop_closest/control/cur_vis_0_1_window_-25cm_0cm
Loaded behav_data_by_bin from all_monkey_data/target_decoder/monkey_Bruno/data_0312/behav_data_by_point.csv
Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
There are 4 duplicated point_index in bin_info. Note: one_point_index_per_bin is False


# FF visible

In [35]:
pn.make_or_retrieve_ff_dataframe()
df = pn.ff_dataframe.copy()

# minimal: detect runs, no merging (each run is its own cluster)
df2 = ff_vis_epochs.compute_visibility_runs_and_clusters(
    df, ff_col='ff_index', t_col='point_index', time_col='time', vis_col='visible',
    chunk_merge_gap=0.05,    # seconds: merge *raw* runs into chunks if gap <= this
    cluster_merge_gap=1
)

df2 = ff_vis_epochs.add_global_visibility_bursts(df2, global_merge_gap=0.25)
#df2 = ff_vis_epochs.add_global_vis_cluster_id(df2, group_cols=None, nullable_int=True)
df2 = ff_vis_epochs.add_global_vis_chunk_id(df2, group_cols=None, nullable_int=True)
#df2 = ff_vis_epochs.add_global_vis_cluster_id(df2, group_cols=None, nullable_int=True)

vis_df = df2.loc[df2['visible'] == 1].copy()


Retrieved ff_dataframe from all_monkey_data/processed_data/monkey_Bruno/data_0312/ff_dataframe.h5
[visibility] raw_runs=9427, merged_into_chunks=174 (1.8%) | chunks=9340, merged_into_clusters=3641 (39.0%)
[visibility][global] total runs=9340, merged into bursts=8787 (94.1%)


In [36]:
# based on any ff visible
sequential_vis_df = vis_df[['ff_index', 'ff_vis_start_time', 'ff_vis_end_time',
                           'global_vis_chunk_id', 'global_burst_id', 'global_burst_start_time','global_burst_end_time',
                           'global_burst_duration','global_burst_size',
                           #'global_burst_prev_start_time','global_burst_prev_end_time'
                           ]].drop_duplicates().reset_index(drop=True)
sequential_vis_df = sequential_vis_df.sort_values('ff_vis_start_time').reset_index(drop=True)
sequential_vis_df['prev_time'] = sequential_vis_df['ff_vis_start_time'].shift(1)
sequential_vis_df['next_time'] = sequential_vis_df['ff_vis_start_time'].shift(-1)


new_seg_info = event_binning.pick_event_window(sequential_vis_df,
                                                event_time_col='ff_vis_start_time',
                                                prev_event_col='prev_time',
                                                next_event_col='next_time',
                                                pre_s=1, post_s=1.0, min_pre_bins=3, min_post_bins=3, bin_dt=0.04)
new_seg_info['event_id'] = new_seg_info['global_vis_chunk_id']
new_seg_info['event_time'] = new_seg_info['ff_vis_start_time']
new_seg_info

,ff_index,ff_vis_start_time,ff_vis_end_time,global_vis_chunk_id,global_burst_id,global_burst_start_time,global_burst_end_time,global_burst_duration,global_burst_size,prev_time,next_time,new_seg_start_time,new_seg_end_time,is_truncated_pre,is_truncated_post,n_pre_bins,n_post_bins,new_seg_duration,ok_window,event_id,event_time
0,93,0.11605,0.11605,1,1,0.11605,2.92143,2.80538,14.00000,NaN,0.13276,-0.88395,0.11605,False,True,25,0,1.00000,False,1,0.11605
1,139,0.13276,0.41491,2,1,0.11605,2.92143,2.80538,14.00000,0.11605,0.26558,0.13276,0.17276,True,True,0,1,0.04000,False,2,0.13276
2,193,0.26558,0.54778,3,1,0.11605,2.92143,2.80538,14.00000,0.13276,0.29921,0.22557,0.26558,True,True,1,0,0.04000,False,3,0.26558
3,152,0.29921,0.58094,4,1,0.11605,2.92143,2.80538,14.00000,0.26558,0.46668,0.29921,0.37921,True,True,0,2,0.08000,False,4,0.29921
4,261,0.46668,0.74689,5,1,0.11605,2.92143,2.80538,14.00000,0.29921,0.87967,0.38667,0.66668,True,True,2,5,0.28000,False,5,0.46668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9335,818,3489.70296,3489.98526,9336,1910,3487.57827,3489.98526,2.40699,12.00000,3489.32172,3491.13063,3489.54296,3490.38296,True,True,4,17,0.84000,True,9336,3489.70296
9336,999,3491.13063,3491.41281,9337,1911,3491.13063,3491.54560,0.41497,2.00000,3489.70296,3491.26343,3490.45063,3491.17063,True,True,17,1,0.72000,False,9337,3491.13063
9337,878,3491.26343,3491.54560,9338,1911,3491.13063,3491.54560,0.41497,2.00000,3491.13063,3491.96058,3491.22343,3491.58343,True,True,1,8,0.36000,False,9338,3491.26343
9338,944,3491.96058,3492.07679,9339,1912,3491.96058,3492.07679,0.11621,2.00000,3491.26343,3492.04360,3491.64058,3492.00058,True,True,8,1,0.36000,False,9339,3491.96058


In [37]:
events_with_stats = sequential_vis_df[['global_vis_chunk_id', 'global_burst_id', 'ff_vis_start_time', 'ff_vis_end_time']].copy()
events_with_stats = sequential_vis_df.rename(columns={'global_vis_chunk_id': 'event_id', 
                                           'global_burst_id': 'event_cluster_id', 
                                           'ff_vis_start_time': 'event_id_start_time', 
                                           'ff_vis_end_time': 'event_id_end_time'})

In [38]:
events_with_stats

,ff_index,event_id_start_time,event_id_end_time,event_id,event_cluster_id,global_burst_start_time,global_burst_end_time,global_burst_duration,global_burst_size,prev_time,next_time
0,93,0.11605,0.11605,1,1,0.11605,2.92143,2.80538,14.00000,NaN,0.13276
1,139,0.13276,0.41491,2,1,0.11605,2.92143,2.80538,14.00000,0.11605,0.26558
2,193,0.26558,0.54778,3,1,0.11605,2.92143,2.80538,14.00000,0.13276,0.29921
3,152,0.29921,0.58094,4,1,0.11605,2.92143,2.80538,14.00000,0.26558,0.46668
4,261,0.46668,0.74689,5,1,0.11605,2.92143,2.80538,14.00000,0.29921,0.87967
...,...,...,...,...,...,...,...,...,...,...,...
9335,818,3489.70296,3489.98526,9336,1910,3487.57827,3489.98526,2.40699,12.00000,3489.32172,3491.13063
9336,999,3491.13063,3491.41281,9337,1911,3491.13063,3491.54560,0.41497,2.00000,3489.70296,3491.26343
9337,878,3491.26343,3491.54560,9338,1911,3491.13063,3491.54560,0.41497,2.00000,3491.13063,3491.96058
9338,944,3491.96058,3492.07679,9339,1912,3491.96058,3492.07679,0.11621,2.00000,3491.26343,3492.04360


# Prepare for GLM

## binned_feats

In [39]:
reload(event_binning)
reload(stop_design)
reload(cluster_design)
reload(design_checks)
reload(stop_glm_fit)
reload(glm_fit_utils)
reload(cv_stop_glm)
reload(plot_tuning_func)

<module 'neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_plotting.plot_tuning_func' from '/Users/dusiyi/Documents/Multifirefly-Project/multiff_analysis/multiff_code/methods/neural_data_analysis/topic_based_neural_analysis/stop_event_analysis/stop_glm/glm_plotting/plot_tuning_func.py'>

In [46]:
# 1) Build bins from your event windows (gaps allowed)
bin_dt = 0.04
bins_2d, meta = event_binning.event_windows_to_bins2d(new_seg_info, bin_dt=bin_dt, only_ok=False)

# 2) Get overlap assignments once
sample_idx, bin_idx_array, dt_array, n_bins = event_binning.build_bin_assignments(
    pn.monkey_information['time'].to_numpy(),
    bins_2d,
)

# 3) Subselect raw samples once
monkey_information_sub = pn.monkey_information.iloc[sample_idx].copy()

# 3a) One pass to get exposure and used_bins
_dummy, exposure, used_bins = event_binning.bin_timeseries_weighted(
    monkey_information_sub['time'].to_numpy(),  # any column of same length works
    dt_array, bin_idx_array, how='mean'
)

# 3b) Aggregate features with the SAME assignments
def agg_feat(col):
    vals = monkey_information_sub[col].to_numpy()
    out, _exp, _ub = event_binning.bin_timeseries_weighted(vals, dt_array, bin_idx_array, how='mean')
    # Defensive checks: exposure/used_bins should match
    assert np.shares_memory(_exp, exposure) or np.allclose(_exp, exposure)
    assert np.array_equal(_ub, used_bins)
    return out

binned_feats = pd.DataFrame({
    'accel':     agg_feat('accel'),
    'speed':     agg_feat('speed'),
    'ang_speed': agg_feat('ang_speed'),
})

# Clean NaNs (optional: choose your policy)
binned_feats = binned_feats.replace([np.inf, -np.inf], np.nan).fillna(0.0)

# 3c) Keep bins with exposure > 0
mask_used = exposure > 0
pos = used_bins[mask_used]
binned_feats = binned_feats.iloc[mask_used].reset_index(drop=True)

meta_by_bin = meta.set_index('bin').sort_index()
meta_used   = meta_by_bin.loc[pos].reset_index()   # rows now match binned_feats 


# 4) Bin spikes per cluster across ALL bins, then slice by pos
spike_counts, cluster_ids = event_binning.bin_spikes_by_cluster(
    pn.spikes_df, bins_2d, time_col='time', cluster_col='cluster'
)

# Sanity checks
assert pos.size == binned_feats.shape[0]
assert spike_counts.shape[0] >= (pos.max() + 1)

binned_spikes = pd.DataFrame(
    spike_counts[pos, :],        # slice rows to align with pos
    columns=cluster_ids,         # cluster IDs as column labels
).reset_index(drop=True)

# Build the stop-aware design block (same helper we wrote earlier)
X_event_df = stop_design.build_event_design_from_meta(
    meta=meta,
    pos=pos,
    new_seg_info=new_seg_info,
    speed_used=binned_feats['speed'].values,
    include_columns=(
        'basis', 'prepost', 'prepost*speed',
        'captured', 'basis*captured', #'prepost*captured',
        #'time_since_prev_event_pre', 'time_to_next_event_post',
        'time_since_prev_event', 'time_to_next_event',
    )
)

cluster_df = cluster_design.build_cluster_features_workflow(
    meta_used[['event_id', 'rel_center']], events_with_stats,
    rel_time_col='rel_center',
    winsor_p=0.5,
    use_midbin_progress=True,
    zscore_progress=False,   # set True if you want progress in SD units
    zscore_rel_time=True
)
cluster_feats = [
        'is_clustered',
        'event_is_first_in_cluster', 
        #'event_is_last_in_cluster',
        'prev_gap_s_z',
        'next_gap_s_z',
        'cluster_duration_s_z',
        'cluster_progress_c', 'cluster_progress_c2',
        #'log_event_cluster_size_z',      # optional
        'cluster_rel_time_s_z',          # optional (bin-level)
    ]


cols_to_add_from_event_design = [c for c in X_event_df.columns if c not in binned_feats.columns]
binned_feats.loc[:, cols_to_add_from_event_design] = X_event_df[cols_to_add_from_event_design].to_numpy()  # equivalent to .values

cols_to_add_from_cluster_design = [c for c in cluster_feats if c not in binned_feats.columns]
binned_feats.loc[:, cols_to_add_from_cluster_design] = cluster_df[cols_to_add_from_cluster_design].to_numpy()  # equivalent to .values

offset_log = np.log(np.clip(exposure[mask_used], 1e-12, None)) 




# --- usage ---
binned_feats_sc, scaled_cols = event_binning.selective_zscore(binned_feats)
binned_feats_sc = sm.add_constant(binned_feats_sc, has_constant='add')
print('Scaled columns:', scaled_cols)

df_X = binned_feats_sc.copy()
df_Y = binned_spikes.copy()


Scaled columns: ['accel', 'speed', 'ang_speed', 'rcos_-0.24s', 'rcos_-0.16s', 'rcos_-0.08s', 'rcos_+0.00s', 'rcos_+0.08s', 'rcos_+0.16s', 'rcos_+0.24s', 'prepost*speed', 'time_since_prev_event', 'time_to_next_event']


In [48]:

# add more columns
binned_feats['time_rel_to_event_start'] = meta_used['rel_center']


# add ff visible/in-memory info
max_in_memory_time_since_seen = 2
ff_df_sub = pn.ff_dataframe[pn.ff_dataframe['time_since_last_vis']
                                        < max_in_memory_time_since_seen].copy()
ff_df_sub['in_memory'] = 1

for state in ['visible', 'in_memory']:
    k_ff_visible = vis_design.count_visible_from_time_df_fast(
        ff_df_sub,
        bins_2d,
        vis_col=state
    )

    any_ff_visible = (k_ff_visible > 0).astype('int8')

    binned_feats[f'any_ff_{state}'] = any_ff_visible[used_bins]
    binned_feats[f'k_ff_{state}'] = k_ff_visible[used_bins]


# --- usage ---
binned_feats_sc, scaled_cols = event_binning.selective_zscore(binned_feats)
binned_feats_sc = sm.add_constant(binned_feats_sc, has_constant='add')
print('Scaled columns:', scaled_cols)

df_X = binned_feats_sc.copy()
df_Y = binned_spikes.copy()

Scaled columns: ['accel', 'speed', 'ang_speed', 'rcos_-0.24s', 'rcos_-0.16s', 'rcos_-0.08s', 'rcos_+0.00s', 'rcos_+0.08s', 'rcos_+0.16s', 'rcos_+0.24s', 'prepost*speed', 'time_since_prev_event', 'time_to_next_event', 'time_rel_to_event_start', 'k_ff_visible', 'k_ff_in_memory']


## fit

In [49]:
reload(glm_fit_utils)
reload(stop_glm_fit)

report = stop_glm_fit.glm_mini_report(
    df_X=df_X, df_Y=df_Y, offset_log=offset_log,
    cov_type='HC1', 
    fast_mle=True,
    do_inference=False, 
    make_plots=True,
    show_plots=True,
)


Skipping terms with no significant clusters (sig_FDR=0): ['next_gap_s_z', 'rcos_-0.24s', 'rcos_-0.16s', 'prepost*speed', 'prepost', 'k_ff_visible', 'cluster_rel_time_s_z', 'time_to_next_event', 'cluster_duration_s_z', 'any_ff_visible']


[glm_mini_report] Finished fast MLE on 25 clusters, cov_type=HC1, inference=False, plots=True
[glm_mini_report] offenders: 1


# Decoding

## run

In [51]:
exposure = np.ones(len(df_X)) * pn.bin_width
offset_log = np.log(exposure)

cols_to_decode = ['any_ff_visible']
glm_decoding.glm_decoding_from_fit(cols_to_decode, df_X, df_Y, offset_log, report)

----------------------------------------------------------------------------------------------------
=== Decoding from fit: any_ff_visible ===
any_ff_visible: mean = 0.511696071247393
AUC=0.572, PR-AUC=0.569


## CV

In [53]:
cols_to_decode = ['any_ff_visible']
groups = meta_used['event_id']
glm_decoding.glm_decoding_cv(cols_to_decode, df_X, df_Y, groups, offset_log)

[glm_mini_report] Finished fast MLE on 25 clusters, cov_type=HC1, inference=False, plots=False
[glm_mini_report] offenders: 1
[glm_mini_report] Finished fast MLE on 25 clusters, cov_type=HC1, inference=False, plots=False
[glm_mini_report] offenders: 0
[glm_mini_report] Finished fast MLE on 25 clusters, cov_type=HC1, inference=False, plots=False
[glm_mini_report] offenders: 1
[glm_mini_report] Finished fast MLE on 25 clusters, cov_type=HC1, inference=False, plots=False
[glm_mini_report] offenders: 1
[glm_mini_report] Finished fast MLE on 25 clusters, cov_type=HC1, inference=False, plots=False
[glm_mini_report] offenders: 1
----------------------------------------------------------------------------------------------------
=== CV decoding: any_ff_visible ===
GroupKFold AUC: 0.563 ± 0.015 | PR-AUC: 0.561 ± 0.017 | folds=5
fold 1: AUC=0.547, PR-AUC=0.546, n_test=14193, kept=14193
fold 2: AUC=0.583, PR-AUC=0.588, n_test=14193, kept=14193
fold 3: AUC=0.556, PR-AUC=0.545, n_test=14193, kept=1

## permutation

In [55]:
cols_to_decode = ['any_ff_visible']
groups = meta_used['event_id']
glm_decoding.glm_decoding_permutation_test(cols_to_decode, df_X, df_Y,
                          groups, offset_log, report)


----------------------------------------------------------------------------------------------------


Permutations:   0%|          | 0/2000 [00:00<?, ?it/s]

=== Permutation test: any_ff_visible ===
Observed AUC        : 0.572
Permutation p-value : 0.0004998  (n=2000)
any_ff_visible: AUC=0.572, p=0.0005, exceed=0/2000, null μ=0.541±0.002
[0.54248 0.54127 0.53811 0.54052 0.54225 0.54209 0.53972 0.54268 0.53874
 0.54038]
